In [1]:
# Uncomment these if running in colab
#!pip install wget
#!pip install newspaper3k
#!pip install xmltodict
#!pip install pandarallel
#!pip install datefinder
#!pip install pydrive
#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [2]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from multiprocessing import Pool
import random
from urllib.parse import quote
import pytz


/opt/ohpc/pub/apps/python/3.8.2/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = "NNA"
country = "Lebanon"
max_click_SHOW_MORE = 500
host = "https://www.nna-leb.gov.lb"
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
urls = [quote('/ar/1/سياسة'),quote('/ar/2/أمن-وقضاء'),quote('/ar/11/دوليات')]
urls = [host + url for url in urls]
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more','scroll_down']
type_of_page = types[0]
extract_text_sleep = random.randint(4,6)
extract_urls_sleep = random.randint(2,4)



# If page type is click_more or scroll_down i.e. can get more article by clicking a button with text 'click more' or simply scrolling down
xpath_for_link = "//h2[contains(@class, 'titulo')]//following::a"
xpath_for_click_more = "//div[contains(@class, 'btn vermasnoticias cf')]"





# If page type is page
target_tag = 'div'
target_tag_class = 'bannerfeatured-item p-relative'

# in case multi tags
# target_tags = ['']
# target_tag_classes = ['']

if type_of_page == 'page':
    urls = [url + '?page=' for url in urls]
pages_each_category = [10,10,10] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 500



In [4]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [5]:

output_path = 'links_'+csv_name

def retrieve_links_from_list(url,scroll_down=False):
  options = FirefoxOptions()
  options.add_argument("--headless")
  driver = webdriver.Firefox(options=options)
  driver.get(url)
  
  count = 0

  news_urls = set()

  prev_length = 0
  while count <= max_click_SHOW_MORE:
      if not scroll_down:
          try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath_for_click_more))
            )
          except TimeoutException as e:
            print('Could not find click more button')
            break
        
    
      try:
          all_links = driver.find_elements(By.XPATH, xpath_for_link)
          if not scroll_down:
            click_more = driver.find_element(By.XPATH, xpath_for_click_more)
          curr_length = len(all_links)
          print(f'{curr_length} links in current page')
          print(f'{prev_length} links in previous page')
          if count > 0:
            if curr_length == prev_length or (not scroll_down and not click_more):
              break
          
          current_links = [l.get_attribute("href") for l in all_links[prev_length:]]
          df_link = pd.DataFrame(current_links, columns = ['link'])
          df_link.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
            
          prev_length = curr_length

          if not scroll_down:
            
            try:
                click_more.click();
            except Exception as e:
                driver.execute_script("arguments[0].click();", click_more) #If click does not work because of overlapping elements, this executes
            
            print(f"Button clicked {count} times", )
          else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"Scrolled down {count+1} times")
          
          time.sleep(3)
            
          count += 1
        
          
          
      except TimeoutException:
          break

  time.sleep(2)

  all_links = driver.find_elements(By.XPATH, xpath_for_link)

  for link in all_links:
        if isinstance(link.get_attribute('href'), str):
          if re.match(relevant_path, link.get_attribute('href')):
            news_urls.add(link.get_attribute('href'))

  driver.quit()
  print('*' * 20)
  return list(news_urls)


def extract_urls(url):    
    rows = set()
    soup = getSoup(url)
    if soup:
      all_elems = None
      if target_tag_class != '':
        all_elems = soup.find_all(target_tag, {'class':target_tag_class})
      else:
        all_elems = soup.find_all(target_tag)
      
      for d in all_elems:
        all_links = d.find_all('a', href=True)
        
        for l in all_links:
            if not re.match('^http.*', l['href']):
              l['href'] = host + l['href']
              
            if re.match(relevant_path, l['href']):
              rows.add(l['href'])
                
    
    if rows:
      return list(rows)
    else:
      return None

def extract_urls_from_df(df):
    links = df['page'].map(lambda x: extract_urls(x))
    df['link'] = links
    return df

In [6]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)
            
elif type_of_page == 'click_more' or type_of_page == 'scroll_down':
    for url in urls:
        pages += retrieve_links_from_list(url, True if type_of_page == 'scroll_down' else False)

        
df = pd.DataFrame(pages, columns = ['page'])
df

,page
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1
1,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=2
2,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=3
3,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=4
4,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=5
5,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=6
6,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=7
7,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=8
8,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=9
9,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=10


In [7]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        req = request.Request(url, headers={'User-Agent' : "Chrome"}) 
        con = request.urlopen(req)
        time.sleep(extract_urls_sleep)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None
        
    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None

    
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [8]:
#New Code caused issues

def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    return df

In [9]:
df

,page
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1
1,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=2
2,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=3
3,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=4
4,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=5
5,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=6
6,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=7
7,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=8
8,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=9
9,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=10


In [10]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:  
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [11]:
if type_of_page == 'page':    
    start = 0
    limit = 40
    total_time_start = time.time()
    results = []
    
    while start < len(df):
        start_time = time.time()
        results.append(parallelize_dataframe(df[start:start+limit], extract_urls_from_df, 90))
        end_time = time.time()
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit
    
    df = pd.concat(results)
    total_time_end = time.time()
    print(f'total time taken: {round(total_time_end - total_time_start,2)} second')

else:
    df = df.rename(columns={"page": "link"})


Batch of data of row range 0-40 complete in 6.71 seconds
100% complete
total time taken: 6.71 second


In [12]:
df

,page,link
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,"[https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87..."
1,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=2,"[https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566717/%D8%A7%D9%84%D8%AA%D9%82%D8%AF%D9%85%D9%8A-%D8%B4%D8%AC%D8%A8-%D9%82%D8%B5%D9%81-%D8%A5%D9%82%D9%84%D9%8A%D9%85-%D9%83%D8%B1%D8%AF%D8%B3%D8%AA%D8%A7%D9%86-%D8%A7%D9%84%D8%B9%D8%B1%D8%A7%D9%82, https://www.nna-leb.gov.lb/ar/gallery/album/2295, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566718/%D8%A7%D9%84%D8%AE%D8%B7%D9%8A%D8%A8-%D9%86%D8%B7%D8%A7%D9%84%D8%A8-%D8%A8%D8%AA%D8%B4%D9%83%D9%8A%D9%84-%D8%AD%D9%83%D9%88%D9%85%D8%A9-%D8%AA%D8%AA%D8%B5%D8%AF%D9%89-%D9%84%D9%84%D8%A7%D9%86%D9%87%D9%8A%D8%A7%D8%B1-..."
2,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=3,"[https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566635/%D8%A7%D9%84%D8%B4%D8%B1%D9%82-%D8%A7%D9%84%D8%AC%D9%84%D8%B3%D8%A9-%D8%A7%D9%84%D8%A3%D9%88%D9%84%D9%89-%D9%81%D8%B1%D8%B7%D8%AA-%D8%A7%D9%84%D9%85%D8%B9%D8%A7%D8%B1%D8%B6%D8%A9-63-%D9%88%D8%B1%D9%82%D8%A9-%D8%A8%D9%8A%D8%B6%D8%A7%D8%A1-%D9%85%D9%86, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566679/%D8%A7%D9%84%D8%B9%D9%84%D8%A7%D9%85%D8%A9-%D9%8A%D8%A7%D8%B3%D9%8A%D9%86-%D8%A7%D9%84%D8%AD%D9%84-%D8%A8%D8%A7%D9%84%D8%A7%D9%86%D8%AA%D9%82%D8%A7%D9%84-%D8%A7%D9%84%D9%89-%D9%86%D8%B8%D8%A7%D9%85-%D8%B3%D9..."
3,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=4,"[https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566578/%D8%B1%D8%A6%D9%8A%D8%B3-%D8%A7%D9%84%D8%AC%D9%85%D9%87%D9%88%D8%B1%D9%8A%D8%A9-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D9%88%D8%B2%D9%8A%D8%B1-%D8%A7%D9%84%D8%B9%D8%AF%D9%84-%D9%85%D8%B3%D8%A7%D8%A6%D9%84-%D9%82%D8%B6%D8%A7%D8%A6%D9%8A%D8%A9-%D8%B9%D8%A7%D9%84%D9%82, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566610/%D9%85%D8%B9%D9%88%D8%B6-%D8%A3%D9%85%D8%AF-%D9%8A%D8%AF%D9%8A-%D8%A7%D9%84%D9%89-%D8%A8%D9%82%D9%8A%D8%A9-%D8%A7%D9%84%D9%85%D8%B9%D8%A7%D8%B1%D8%B6%D8%A9-%D9%88%D8%AE%D9%8A%D8%A7%D8%B1%D9%8A-%D8%..."
4,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=5,"[https://www.nna-leb.gov.lb/ar/gallery/album/2295, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566563/%D9%82%D8%A7%D8%B3%D9%85-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%88%D9%81%D8%AF%D8%A7-%D9%82%D9%8A%D8%A7%D8%AF%D9%8A%D8%A7-%D9%85%D9%86-%D8%A7%D9%84%D8%AC%D8%A8%D9%87%D8%A9-%D8%A7%D9%84%D8%B4%D8%B9%D8%A8%D9%8A%D8%A9-%D8%A7%D9%84%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566569/%D9%82%D8%A7%D8%A6%D8%AF-%D8%A7%D9%84%D8%AC%D9%8A%D8%B4-%D8%A8%D8%AD%D8%AB-%D9%85%D8%B9-%D9%82%D8%A7%D8%A6%D8%AF-%D8%A7%D9%84%D9%82%D9%88%D8%A..."
5,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=6,"[https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566471/%D8%B3%D9%84%D8%A7%D9%85-%D8%A7%D8%A8%D8%B1%D9%82-%D8%A7%D9%84%D9%89-%D8%A7%D9%84%D8%B9%D8%A7%D9%87%D9%84-%D8%A7%D9%84%D8%B3%D8%B9%D9%88%D8%AF%D9%8A-%D9%88%D9%88%D9%84%D9%8A-%D8%A7%D9%84%D8%B9%D9%87%D8%AF-%D9%85%D9%87%D9%86%D8%A6%D8%A7-%D8%A8%D8%AA%D8%B9%D9%8A, https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566506/%D8%A7%D9%84%D8%AE%D8%B7%D9%8A%D8%A8-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D8%A7%D9%85%D9%8A%D9%84-%D8%B1%D8%AD%D9%85%D9%87-%D8%A7%D9%84%D9%84%D8%A

In [13]:
df.to_csv('links_'+csv_name)

In [14]:
print(len(df))
df = df[df['link'].notna()]
df = df[(df['link'].str.len() != 0)]
print(len(df))

30
30


In [15]:
if type_of_page == 'page':
    df = df.explode('link')
df

,page,link
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D8%A7%D9%84
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566798/%D9%81%D8%B1%D9%86%D8%AC%D9%8A%D9%87-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1%D8%A9-%D8%A7%D9%84%D9%81%D8%B1%D9%86%D8%B3%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%88%D8%B6%D8%A7%D8%B9-%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9-%D9%88%D8%A7%D9%84
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/gallery/album/2295
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566793/%D8%B1%D8%A6%D9%8A%D8%B3-%D8%A7%D9%84%D8%AC%D9%85%D9%87%D9%88%D8%B1%D9%8A%D8%A9-%D8%B9%D8%B1%D8%B6-%D8%B9%D9%88%D8%AF%D8%A9-%D8%A7%D9%84%D9%86%D8%A7%D8%B2%D8%AD%D9%8A%D9%86-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1-%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A
...,...,...
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/565161/%D9%87%D8%B2%D8%A9-%D8%A3%D8%B1%D8%B6%D9%8A%D8%A9-%D8%A8%D9%82%D9%88%D8%A9-4-4-%D8%AF%D8%B1%D8%AC%D8%A7%D8%AA-%D8%B6%D8%B1%D8%A8%D8%AA-%D9%85%D8%AF%D9%8A%D9%86%D8%A9-%D8%AC%D9%86%D9%88%D8%A9-%D8%A7%D9%84%D8%A7%D9%8A%D8%B7%D8%A7%D9%84%D9%8A%D8%A9
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/564984/%D9%84%D8%A7%D9%81%D8%B1%D9%88%D9%81-%D8%B9%D9%82%D8%AF-%D9%84%D9%82%D8%A7%D8%A1%D8%A7%D8%AA-%D9%85%D9%83%D8%AB%D9%81%D8%A9-%D8%B9%D9%84%D9%89-%D9%87%D8%A7%D9%85%D8%B4-%D9%81%D8%B9%D8%A7%D9%84%D9%8A%D8%A7%D8%AA-%D8%A7%D9%84%D8%AC%D9%85%D8%B9%D9%8A%D8%A9-%D8%A7
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/565066/%D9%85%D9%8A%D8%AF%D9%81%D9%8A%D8%AF%D9%8A%D9%81-%D8%AC%D9%85%D9%8A%D8%B9-%D8%A7%D9%84%D8%A3%D8%B3%D9%84%D8%AD%D8%A9-%D8%A8%D9%85%D8%A7-%D9%81%D9%8A-%D8%B0%D9%84%D9%83-%D8%A7%D9%84%D9%86%D9%88%D9%88%D9%8A%D8%A9-%D8%B3%D8%AA%D8%B3%D8%AA%D8%AE%D8%AF%D9%85-%D9%84
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/564988/%D8%A8%D9%83%D9%8A%D9%86-%D8%A3%D9%85%D9%8A%D8%B1%D9%83%D8%A7-%D8%AA%D8%B3%D8%B1%D9%82-82-%D9%85%D9%86-%D9%86%D9%81%D8%B7-%D8%B3%D9%88%D8%B1%D9%8A%D8%A7-%D9%88%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A%D9%88%D9%86-%D9%8A%D8%B5%D8%B7%D9%81%D9%88%D9%86


In [16]:
print(len(df))
df = df.drop_duplicates(subset='link')
print(len(df))

510
481


In [17]:
#Create a links checkpoint csv
df.to_csv('links_'+csv_name)
df.to_csv('/xdisk/josorio1/salsarra/links/'+'links_'+csv_name, encoding = 'utf-8-sig')

In [18]:
if 'df' not in locals() or df is None or 'link' not in df:
    if(os.path.exists('links_'+csv_name)):
        df = pd.read_csv('links_'+csv_name, index_col=0)

In [19]:
df

,page,link
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D8%A7%D9%84
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566798/%D9%81%D8%B1%D9%86%D8%AC%D9%8A%D9%87-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1%D8%A9-%D8%A7%D9%84%D9%81%D8%B1%D9%86%D8%B3%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%88%D8%B6%D8%A7%D8%B9-%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9-%D9%88%D8%A7%D9%84
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/gallery/album/2295
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566793/%D8%B1%D8%A6%D9%8A%D8%B3-%D8%A7%D9%84%D8%AC%D9%85%D9%87%D9%88%D8%B1%D9%8A%D8%A9-%D8%B9%D8%B1%D8%B6-%D8%B9%D9%88%D8%AF%D8%A9-%D8%A7%D9%84%D9%86%D8%A7%D8%B2%D8%AD%D9%8A%D9%86-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1-%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A
...,...,...
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/565161/%D9%87%D8%B2%D8%A9-%D8%A3%D8%B1%D8%B6%D9%8A%D8%A9-%D8%A8%D9%82%D9%88%D8%A9-4-4-%D8%AF%D8%B1%D8%AC%D8%A7%D8%AA-%D8%B6%D8%B1%D8%A8%D8%AA-%D9%85%D8%AF%D9%8A%D9%86%D8%A9-%D8%AC%D9%86%D9%88%D8%A9-%D8%A7%D9%84%D8%A7%D9%8A%D8%B7%D8%A7%D9%84%D9%8A%D8%A9
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/564984/%D9%84%D8%A7%D9%81%D8%B1%D9%88%D9%81-%D8%B9%D9%82%D8%AF-%D9%84%D9%82%D8%A7%D8%A1%D8%A7%D8%AA-%D9%85%D9%83%D8%AB%D9%81%D8%A9-%D8%B9%D9%84%D9%89-%D9%87%D8%A7%D9%85%D8%B4-%D9%81%D8%B9%D8%A7%D9%84%D9%8A%D8%A7%D8%AA-%D8%A7%D9%84%D8%AC%D9%85%D8%B9%D9%8A%D8%A9-%D8%A7
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/565066/%D9%85%D9%8A%D8%AF%D9%81%D9%8A%D8%AF%D9%8A%D9%81-%D8%AC%D9%85%D9%8A%D8%B9-%D8%A7%D9%84%D8%A3%D8%B3%D9%84%D8%AD%D8%A9-%D8%A8%D9%85%D8%A7-%D9%81%D9%8A-%D8%B0%D9%84%D9%83-%D8%A7%D9%84%D9%86%D9%88%D9%88%D9%8A%D8%A9-%D8%B3%D8%AA%D8%B3%D8%AA%D8%AE%D8%AF%D9%85-%D9%84
29,https://www.nna-leb.gov.lb/ar/11/%D8%AF%D9%88%D9%84%D9%8A%D8%A7%D8%AA?page=10,https://www.nna-leb.gov.lb/ar/international/564988/%D8%A8%D9%83%D9%8A%D9%86-%D8%A3%D9%85%D9%8A%D8%B1%D9%83%D8%A7-%D8%AA%D8%B3%D8%B1%D9%82-82-%D9%85%D9%86-%D9%86%D9%81%D8%B7-%D8%B3%D9%88%D8%B1%D9%8A%D8%A7-%D9%88%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A%D9%88%D9%86-%D9%8A%D8%B5%D8%B7%D9%81%D9%88%D9%86


In [20]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 40

res = []
while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 40)
    res.append(test_df)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
    test_df['text_set'] = test_df['content'].map(lambda x: True if x[4] != '' else False)    
    no_of_text_retrieved = test_df.text_set.sum()
    print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')
    
    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit

df = pd.concat(res)

39 / 40 text retrieved
Batch of data of row range 0-40 complete in 9.09 seconds
8.32% complete
40 / 40 text retrieved
Batch of data of row range 40-80 complete in 9.57 seconds
16.63% complete
33 / 40 text retrieved
Batch of data of row range 80-120 complete in 11.15 seconds
24.95% complete
40 / 40 text retrieved
Batch of data of row range 120-160 complete in 9.42 seconds
33.26% complete
39 / 40 text retrieved
Batch of data of row range 160-200 complete in 8.58 seconds
41.58% complete
40 / 40 text retrieved
Batch of data of row range 200-240 complete in 9.41 seconds
49.9% complete
40 / 40 text retrieved
Batch of data of row range 240-280 complete in 9.37 seconds
58.21% complete
40 / 40 text retrieved
Batch of data of row range 280-320 complete in 11.15 seconds
66.53% complete
40 / 40 text retrieved
Batch of data of row range 320-360 complete in 11.39 seconds
74.84% complete
40 / 40 text retrieved
Batch of data of row range 360-400 complete in 10.9 seconds
83.16% complete
40 / 40 text re

In [21]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
#         df = pd.read_csv(output_path, on_bad_lines='skip', converters={'content': pd.eval}, index_col=0)
        df = pd.read_csv(output_path, on_bad_lines='skip', index_col=0)

In [22]:
df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)     
df


,page,link,content,text_set,country,news_outlet,title,date,text
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9,"[Lebanon, NNA, الوكالة الوطنية للإعلام - البعريني استقبل معوّض في القموعة وتأكيد على ضرورة احتضان لبنان عربيًا ودوليًا, , وطنية - عكار - استقبل عضو تكتل ""الاعتدال الوطني"" وليد البعريني وعقيلته رئيس حركة ""الإستقلال"" النائب ميشال معوض وعقيلته وفريق عمل مؤسسة رينيه معوض، في دارته في فنيدق، بحضور عدد من فاعليات البلدة وأهال.\n\nوقال البعريني: ""علاقتنا مع آل معوض ليست بجديدة، فقد بدأت خلال انتخابات الشمال قبل أن يتم تعديل قانون الإنتخاب، وبالرغم من خسارتنا للعلاقة الإنتخابية بحسب قانون الإنتخاب المُستحدث، لكننا لا نزال محافظين على صداقتنا وتواصلنا مع آل معوض برئاسة النائب الصديق ميشال، فأهلاً و...",True,Lebanon,NNA,الوكالة الوطنية للإعلام - البعريني استقبل معوّض في القموعة وتأكيد على ضرورة احتضان لبنان عربيًا ودوليًا,,"وطنية - عكار - استقبل عضو تكتل ""الاعتدال الوطني"" وليد البعريني وعقيلته رئيس حركة ""الإستقلال"" النائب ميشال معوض وعقيلته وفريق عمل مؤسسة رينيه معوض، في دارته في فنيدق، بحضور عدد من فاعليات البلدة وأهال.\n\nوقال البعريني: ""علاقتنا مع آل معوض ليست بجديدة، فقد بدأت خلال انتخابات الشمال قبل أن يتم تعديل قانون الإنتخاب، وبالرغم من خسارتنا للعلاقة الإنتخابية بحسب قانون الإنتخاب المُستحدث، لكننا لا نزال محافظين على صداقتنا وتواصلنا مع آل معوض برئاسة النائب الصديق ميشال، فأهلاً وسهلاً به وبالوفد المرافق في دارتهم الثانية"".\n\nأضاف: ""لقد تم التحضير لهذه الزيارة قبل تعيين موعد جلسة الانتخاب الأولى لرئ..."
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D8%A7%D9%84,"[Lebanon, NNA, الوكالة الوطنية للإعلام - عبدالرحيم مراد: استخراج الغاز سيغير وجه لبنان المالي والبت في الترسيم خلال الأيام المقبلة, 2035-09-30, وطنية - أشاد رئيس الجامعة اللبنانية الدولية النائب السابق عبد الرحيم مراد، خلال لقاء تربوي عشية بدء العام الجامعي، بموظفي الجامعة الذين ""عملوا ويعملون في جد لإنجاح الجامعة والحفاظ على سمعتها حتى انتشرت في غضون أقل من ٢٢ عاما في لبنان والعالم العربي، فوصل عدد طلابها إلى ٣٥ ألف طالب، واعدا بافتتاح المزيد من الصروح التعليمية عربيا، وتحديدا في غزة في فلسطين المحتلة.\n\nوعن زيادة الرواتب قال: ""عملنا في السابق على رفعها نسبيا، واليوم نحاول ذلك بالقدر الم...",True,Lebanon,NNA,الوكالة الوطنية للإعلام - عبدالرحيم مراد: استخراج الغاز سيغير وجه لبنان المالي والبت في الترسيم خلال الأيام المقبلة,2035-09-30,"وطنية - أشاد رئيس الجامعة اللبنانية الدولية النائب السابق عبد الرحيم مراد، خلال لقاء تربوي عشية بدء العام الجامعي، بموظفي الجامعة الذين ""عملوا ويعملون في جد لإنجاح الجامعة والحفاظ على سمعتها حتى انتشرت في غضون أقل من ٢٢ عاما في لبنان والعالم العربي، فوصل عدد طلابها إلى ٣٥ ألف طالب، واعدا بافتتاح المزيد من الصروح التعليمية عربيا، وتحديدا في غزة في فلسطين المحتلة.\n\nوعن زيادة الرواتب قال: ""عملنا في السابق على رفعها نسبيا، واليوم نحاول ذلك بالقدر الممكن، ولا يكلف الله نفسا إلا وسعها""، معلنًا عن حاجة المؤسسات خارج لبنان إلى أساتذة للعمل فيها.\n\nوعن ترسيم الحدود مع فلسطين، أشار مراد إلى أن ""ل..."
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566798/%D9%81%D8%B1%D9%86%D8%AC%D9%8A%D9%87-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1%D8%A9-%D8%A7%D9%84%D9%81%D8%B1%D9%86%D8%B3%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%88%D8%B6%D8%A7%D8%B9-%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9-%D9%88%D8%A7%D9%84,"[Lebanon, NNA, فرنجيه عرض مع السفيرة

In [23]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

481
472


,page,link,content,text_set,country,news_outlet,title,date,text
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9,"[Lebanon, NNA, الوكالة الوطنية للإعلام - البعريني استقبل معوّض في القموعة وتأكيد على ضرورة احتضان لبنان عربيًا ودوليًا, , وطنية - عكار - استقبل عضو تكتل ""الاعتدال الوطني"" وليد البعريني وعقيلته رئيس حركة ""الإستقلال"" النائب ميشال معوض وعقيلته وفريق عمل مؤسسة رينيه معوض، في دارته في فنيدق، بحضور عدد من فاعليات البلدة وأهال.\n\nوقال البعريني: ""علاقتنا مع آل معوض ليست بجديدة، فقد بدأت خلال انتخابات الشمال قبل أن يتم تعديل قانون الإنتخاب، وبالرغم من خسارتنا للعلاقة الإنتخابية بحسب قانون الإنتخاب المُستحدث، لكننا لا نزال محافظين على صداقتنا وتواصلنا مع آل معوض برئاسة النائب الصديق ميشال، فأهلاً و...",True,Lebanon,NNA,الوكالة الوطنية للإعلام - البعريني استقبل معوّض في القموعة وتأكيد على ضرورة احتضان لبنان عربيًا ودوليًا,,"وطنية - عكار - استقبل عضو تكتل ""الاعتدال الوطني"" وليد البعريني وعقيلته رئيس حركة ""الإستقلال"" النائب ميشال معوض وعقيلته وفريق عمل مؤسسة رينيه معوض، في دارته في فنيدق، بحضور عدد من فاعليات البلدة وأهال.\n\nوقال البعريني: ""علاقتنا مع آل معوض ليست بجديدة، فقد بدأت خلال انتخابات الشمال قبل أن يتم تعديل قانون الإنتخاب، وبالرغم من خسارتنا للعلاقة الإنتخابية بحسب قانون الإنتخاب المُستحدث، لكننا لا نزال محافظين على صداقتنا وتواصلنا مع آل معوض برئاسة النائب الصديق ميشال، فأهلاً وسهلاً به وبالوفد المرافق في دارتهم الثانية"".\n\nأضاف: ""لقد تم التحضير لهذه الزيارة قبل تعيين موعد جلسة الانتخاب الأولى لرئ..."
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D8%A7%D9%84,"[Lebanon, NNA, الوكالة الوطنية للإعلام - عبدالرحيم مراد: استخراج الغاز سيغير وجه لبنان المالي والبت في الترسيم خلال الأيام المقبلة, 2035-09-30, وطنية - أشاد رئيس الجامعة اللبنانية الدولية النائب السابق عبد الرحيم مراد، خلال لقاء تربوي عشية بدء العام الجامعي، بموظفي الجامعة الذين ""عملوا ويعملون في جد لإنجاح الجامعة والحفاظ على سمعتها حتى انتشرت في غضون أقل من ٢٢ عاما في لبنان والعالم العربي، فوصل عدد طلابها إلى ٣٥ ألف طالب، واعدا بافتتاح المزيد من الصروح التعليمية عربيا، وتحديدا في غزة في فلسطين المحتلة.\n\nوعن زيادة الرواتب قال: ""عملنا في السابق على رفعها نسبيا، واليوم نحاول ذلك بالقدر الم...",True,Lebanon,NNA,الوكالة الوطنية للإعلام - عبدالرحيم مراد: استخراج الغاز سيغير وجه لبنان المالي والبت في الترسيم خلال الأيام المقبلة,2035-09-30,"وطنية - أشاد رئيس الجامعة اللبنانية الدولية النائب السابق عبد الرحيم مراد، خلال لقاء تربوي عشية بدء العام الجامعي، بموظفي الجامعة الذين ""عملوا ويعملون في جد لإنجاح الجامعة والحفاظ على سمعتها حتى انتشرت في غضون أقل من ٢٢ عاما في لبنان والعالم العربي، فوصل عدد طلابها إلى ٣٥ ألف طالب، واعدا بافتتاح المزيد من الصروح التعليمية عربيا، وتحديدا في غزة في فلسطين المحتلة.\n\nوعن زيادة الرواتب قال: ""عملنا في السابق على رفعها نسبيا، واليوم نحاول ذلك بالقدر الممكن، ولا يكلف الله نفسا إلا وسعها""، معلنًا عن حاجة المؤسسات خارج لبنان إلى أساتذة للعمل فيها.\n\nوعن ترسيم الحدود مع فلسطين، أشار مراد إلى أن ""ل..."
0,https://www.nna-leb.gov.lb/ar/1/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9?page=1,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566798/%D9%81%D8%B1%D9%86%D8%AC%D9%8A%D9%87-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1%D8%A9-%D8%A7%D9%84%D9%81%D8%B1%D9%86%D8%B3%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%88%D8%B6%D8%A7%D8%B9-%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9-%D9%88%D8%A7%D9%84,"[Lebanon, NNA, فرنجيه عرض مع السفيرة

In [24]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)

In [25]:
df

,link,text_set,country,news_outlet,title,date,text
0,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566813/%D8%A7%D9%84%D8%A8%D8%B9%D8%B1%D9%8A%D9%86%D9%8A-%D8%A7%D8%B3%D8%AA%D9%82%D8%A8%D9%84-%D9%85%D8%B9%D9%88-%D8%B6-%D9%81%D9%8A-%D8%A7%D9%84%D9%82%D9%85%D9%88%D8%B9%D8%A9-%D9%88%D8%AA%D8%A3%D9%83%D9%8A%D8%AF-%D8%B9%D9%84%D9%89-%D8%B6%D8%B1%D9%88%D8%B1%D8%A9,True,Lebanon,NNA,الوكالة الوطنية للإعلام - البعريني استقبل معوّض في القموعة وتأكيد على ضرورة احتضان لبنان عربيًا ودوليًا,,"وطنية - عكار - استقبل عضو تكتل ""الاعتدال الوطني"" وليد البعريني وعقيلته رئيس حركة ""الإستقلال"" النائب ميشال معوض وعقيلته وفريق عمل مؤسسة رينيه معوض، في دارته في فنيدق، بحضور عدد من فاعليات البلدة وأهال.\n\nوقال البعريني: ""علاقتنا مع آل معوض ليست بجديدة، فقد بدأت خلال انتخابات الشمال قبل أن يتم تعديل قانون الإنتخاب، وبالرغم من خسارتنا للعلاقة الإنتخابية بحسب قانون الإنتخاب المُستحدث، لكننا لا نزال محافظين على صداقتنا وتواصلنا مع آل معوض برئاسة النائب الصديق ميشال، فأهلاً وسهلاً به وبالوفد المرافق في دارتهم الثانية"".\n\nأضاف: ""لقد تم التحضير لهذه الزيارة قبل تعيين موعد جلسة الانتخاب الأولى لرئ..."
0,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566836/%D8%B9%D8%A8%D8%AF%D8%A7%D9%84%D8%B1%D8%AD%D9%8A%D9%85-%D9%85%D8%B1%D8%A7%D8%AF-%D8%A7%D8%B3%D8%AA%D8%AE%D8%B1%D8%A7%D8%AC-%D8%A7%D9%84%D8%BA%D8%A7%D8%B2-%D8%B3%D9%8A%D8%BA%D9%8A%D8%B1-%D9%88%D8%AC%D9%87-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A7%D9%84%D9%85%D8%A7%D9%84,True,Lebanon,NNA,الوكالة الوطنية للإعلام - عبدالرحيم مراد: استخراج الغاز سيغير وجه لبنان المالي والبت في الترسيم خلال الأيام المقبلة,2035-09-30,"وطنية - أشاد رئيس الجامعة اللبنانية الدولية النائب السابق عبد الرحيم مراد، خلال لقاء تربوي عشية بدء العام الجامعي، بموظفي الجامعة الذين ""عملوا ويعملون في جد لإنجاح الجامعة والحفاظ على سمعتها حتى انتشرت في غضون أقل من ٢٢ عاما في لبنان والعالم العربي، فوصل عدد طلابها إلى ٣٥ ألف طالب، واعدا بافتتاح المزيد من الصروح التعليمية عربيا، وتحديدا في غزة في فلسطين المحتلة.\n\nوعن زيادة الرواتب قال: ""عملنا في السابق على رفعها نسبيا، واليوم نحاول ذلك بالقدر الممكن، ولا يكلف الله نفسا إلا وسعها""، معلنًا عن حاجة المؤسسات خارج لبنان إلى أساتذة للعمل فيها.\n\nوعن ترسيم الحدود مع فلسطين، أشار مراد إلى أن ""ل..."
0,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566798/%D9%81%D8%B1%D9%86%D8%AC%D9%8A%D9%87-%D8%B9%D8%B1%D8%B6-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1%D8%A9-%D8%A7%D9%84%D9%81%D8%B1%D9%86%D8%B3%D9%8A%D8%A9-%D8%A7%D9%84%D8%A7%D9%88%D8%B6%D8%A7%D8%B9-%D8%A7%D9%84%D9%85%D8%AD%D9%84%D9%8A%D8%A9-%D9%88%D8%A7%D9%84,True,Lebanon,NNA,فرنجيه عرض مع السفيرة الفرنسية الاوضاع المحلية والاقليمية والدولية,,"وطنية - استقبل رئيس ""تيار المرده"" سليمان فرنجيه في دارته في بنشعي السفيرة الفرنسية في لبنان آن غريو، في حضور المستشارين كانتان جانتي ورومان كالفاري والوزير السابق روني عريجي.\n\n\n\nوكان بحث في مجمل التطورات الاقليمية والدولية والمستجدات المحلية لا سيما المتعلق منها بجلسة الأمس النيابية.\n\n\n\n====================="
0,https://www.nna-leb.gov.lb/ar/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9/566793/%D8%B1%D8%A6%D9%8A%D8%B3-%D8%A7%D9%84%D8%AC%D9%85%D9%87%D9%88%D8%B1%D9%8A%D8%A9-%D8%B9%D8%B1%D8%B6-%D8%B9%D9%88%D8%AF%D8%A9-%D8%A7%D9%84%D9%86%D8%A7%D8%B2%D8%AD%D9%8A%D9%86-%D9%85%D8%B9-%D8%A7%D9%84%D8%B3%D9%81%D9%8A%D8%B1-%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A,True,Lebanon,NNA,رئيس الجمهورية عرض عودة النازحين مع السفير السوري وأوضاعا تربوية مع طرابلسي,,"وطنية - عرض رئيس الجمهورية العماد ميشال عون، قبل ظهر اليوم، في قصر بعبدا، مع السفير السوري علي عبد الكريم علي، العلاقات الثنائية بين لبنان وسوريا، لا سيما ما يتصل بأوضاع النازحين السوريين في لبنان في ضوء الخطة التي يضعها لبنان من أجل إعادتهم تدريجيا إلى بلادهم.\n\nوبعد اللقاء، أدلى السفير السوري بتصريح قال فيه: ""سعدت اليوم بمقابلة فخامة الرئيس ونقلت إليه تحيات سيادة الرئيس بشار الأسد وتمنياته للبنان بدوام التقدم. وتطرقنا خلال اللقاء إلى عدد من المواضيع التي تهم البلدين، انطلاقا من العلاقات الأخوية التي تجمع بيننا، خصوصا مسألة عودة النازحين السوريين من لبنان إلى سوريا، والتي نشهد اليوم أكثر..."
0,https://www.nna-leb.gov.lb/ar/%D8%B

In [26]:
tz_DFW = pytz.timezone('US/Central') 
current_time = datetime.now(tz_DFW)
time = current_time.strftime("(%H:%M:%S)")

df.to_csv('/xdisk/josorio1/salsarra/csv/'+time+csv_name, encoding = 'utf-8-sig')

#df.to_csv(time+csv_name)

In [27]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
